In [1]:
from shakemap_lookup import usgs_web

In [2]:
search_params = {
    'starttime':  "1900-01-01",
    'endtime':  "2020-01-01",
    'minmagnitude':  6.5,
    'maxmagnitude': 10.0,
    'mindepth': 0.0,
    'maxdepth': 50.0,
    'minlongitude': -180.0,
    'maxlongitude': 180.0,
    'minlatitude':  -90.0,
    'maxlatitude': 90.0,
    'limit': 10000,
    'producttype': 'shakemap'
}

events = usgs_web.search_usgsevents(search_params)

Sending query to get events...
Parsing...
	...1216 events returned (limit of 10000)
		 60006bjl : M 7.1 - 138km E of Bitung, Indonesia
		 7000635e : M 6.6 - 136km W of Neiafu, Tonga
		 700061e9 : M 6.5 - 2km E of Bulatukan, Philippines
		 6000645n : M 6.6 - 8km NE of Columbio, Philippines
		 70005nie : M 6.7 - 69km WSW of Constitucion, Chile
		 70005lfd : M 6.5 - 11km S of Kairatu, Indonesia
		 700058yy : M 6.6 - 131km S of Bristol Island, South Sandwich Islands
		 60004zhq : M 6.9 - 106km WSW of Tugu Hilir, Indonesia
		 60004yps : M 6.8 - 95km SW of San Antonio, Chile
		 70004jyv : M 7.2 - 93km NNE of Laiwui, Indonesia
		 70004jxe : M 6.6 - 200km W of Broome, Australia
		 70004dz3 : M 6.9 - 134km WSW of Kota Ternate, Indonesia
		 38457511 : M 7.1 - 2019 Ridgecrest Earthquake Sequence
		 6000417i : M 7.3 - 116km NE of L'Esperance Rock, New Zealand
		 70003kyy : M 7.6 - 46km SSE of Namatanai, Papua New Guinea
		 700034xq : M 6.8 - 98km SSW of Luwuk, Indonesia
		 600032fg : M 6.5 - 83km 

In [3]:
for k, v in events['metadata'].items():
    print(k,":", v)

generated : 1576187542000
url : https://earthquake.usgs.gov/fdsnws/event/1/query?starttime=1900-01-01&endtime=2020-01-01&minmagnitude=6.5&maxmagnitude=10.0&mindepth=0.0&maxdepth=50.0&minlongitude=-180.0&maxlongitude=180.0&minlatitude=-90.0&maxlatitude=90.0&limit=10000&producttype=shakemap&format=geojson&jsonerror=true
title : USGS Earthquakes
status : 200
api : 1.8.1
limit : 10000
offset : 1
count : 1216


In [4]:
# Format into a dataframe
import pandas as pd

# Extract info per event
event_info = pd.DataFrame([f['properties'] for f in events['features']])

# Extract centroid locations
event_geom = pd.DataFrame([f['geometry'] for f in events['features']])

# Combine
event_table = pd.concat([event_info, event_geom], axis=1)

# Set ID
event_table['id'] = [ev['id'] for ev in events['features']]
event_table = event_table.set_index('id', verify_integrity=True)

print(len(event_table))
event_table.columns


1216


Index(['mag', 'place', 'time', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi',
       'mmi', 'alert', 'status', 'tsunami', 'sig', 'net', 'code', 'ids',
       'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'type',
       'title', 'type', 'coordinates'],
      dtype='object')

In [6]:
# Get the shakemap details per event
from tqdm import tqdm
import requests
import json

details = []

for url in tqdm(event_table['detail']):
    resp2 = requests.get(url)
    detail = json.loads(resp2.content)
    details.append(detail['properties']['products']['shakemap'])

len(details)

100%|██████████| 1216/1216 [18:52<00:00,  1.07it/s]


1216

In [17]:
count = pd.Series([len(x) for x in details], index=event_table.index)
count.value_counts().to_frame()

,0
1,1116
2,94
3,6


In [8]:
with open('shakemap_details.json', 'w') as fout:
    json.dump(details, fout)

In [21]:
event_table['count'] = count
event_table.loc[count==3, ['mag', 'title', 'time']]

,mag,title,time
id,,,
ci38457511,7.1,M 7.1 - 2019 Ridgecrest Earthquake Sequence,1562383193040
nc72182046,6.8,"M 6.8 - 78km WNW of Ferndale, California",1394428693430
usb000jgju,6.5,"M 6.5 - 81km SSW of Atka, Alaska",1378261950600
usb000jdt7,7.0,"M 7.0 - 101km SW of Atka, Alaska",1377879902300
usc000hrnr,6.5,"M 6.5 - 46km W of Masachapa, Nicaragua",1371317667810
official20120411083836720_20,8.6,M 8.6 - off the west coast of northern Sumatra,1334133516720
